# Proyecto IA Pokémon: Entrenador Virtual basado en Combates Reales

Este proyecto tiene como objetivo desarrollar una inteligencia artificial capaz de jugar combates Pokémon de forma autónoma, utilizando datos reales extraídos de simulaciones del entorno competitivo Pokémon Showdown (Gen9 Random Battles).

A partir de los logs JSON de combates reales, construiremos un pipeline que permita:

- Cargar y procesar los combates turno a turno  
- Extraer características relevantes del estado del combate (tipo, vida, estado, movimientos disponibles, etc.)
- Entrenar modelos de machine learning (aprendizaje supervisado o por refuerzo) para predecir el mejor movimiento
- Simular combates contra el usuario o contra otras IAs

Este notebook será el punto de partida para el desarrollo de la IA, comenzando por la exploración del dataset, limpieza, creación de características y entrenamiento de un primer modelo básico.

In [11]:
# Manejo de archivos y rutas
import json
from pathlib import Path

# Análisis de datos
import pandas as pd
import numpy as np
from pprint import pprint

# barra de progreso si se desea
from tqdm import tqdm

## Consolidación de archivos JSON de batallas

Los datos originales se encuentran distribuidos en múltiples archivos `.json`, cada uno correspondiente a un combate diferente de Pokémon Showdown.

Este bloque de código tiene como objetivo:

- Leer todos los archivos `.json` ubicados en la carpeta `data/battles/`.
- Cargar cada combate en una lista de Python (`battles_data`).
- Unificar todos los combates en un único archivo llamado `all_battles.json`.
- Dejarlo listo para ser utilizado en las siguientes etapas del proyecto (análisis, extracción de features, modelado, etc.).

Esta estrategia nos permite trabajar de forma más eficiente con un único archivo en memoria, facilitando tanto la exploración como el procesamiento masivo de datos.

El archivo resultante `data/all_battles.json` contiene una lista de objetos JSON, donde cada objeto representa un combate completo.

In [ ]:
# Ruta al directorio JSON
battles_dir = Path("data/battles")

# Comprobaciones básicas
assert battles_dir.exists() and battles_dir.is_dir(), f"No existe el directorio: {battles_dir.resolve()}"

# Obtener todos los archivos .json en esa carpeta
json_files = sorted(battles_dir.glob("*.json"))

print(f"Archivos encontrados: {len(json_files)}")
# Muestra algunos ejemplos
for p in json_files[:3]:
    print(f"- {p.name}")

Archivos encontrados: 13979


In [ ]:
# Lista para almacenar todos los combates
battles_data = []

errores = 0
for file in json_files:
    try:
        with open(file, "r") as f:
            battle = json.load(f)
            battles_data.append(battle)
    except Exception as e:
        errores += 1
        print(f"[WARN] Error al procesar {file.name}: {e}")

# Guardar la lista completa en un nuevo archivo JSON
output_path = Path("data/all_battles.json")
with open(output_path, "w") as f:
    json.dump(battles_data, f, indent=2)

print(f"Archivo consolidado guardado en: {output_path.resolve()}")
print(f"Batallas válidas: {len(battles_data)} | Errores: {errores}")

Archivo consolidado guardado en: /Users/alexg.herrera/Desktop/HackABoss/Pokemon_battle/data/all_battles.json


## Validaciones y control de calidad

Antes de continuar con EDA/modelado:
- Verificamos que el número de combates cargados coincide razonablemente con los archivos válidos.
- Inspeccionamos claves esperadas a alto nivel (p. ej., `players`, `turns`, `winner` o equivalentes).
- Calculamos estadísticas simples (número de turnos por combate, presencia de ganador).

In [6]:
df = pd.read_json("data/all_battles.json")
print(df.shape)
df.head()

(13979, 9)


,schema_version,battle_id,format_id,metadata,players,team_revelation,turns,summary,integrity
0,1.0.0,gen9randombattle-2400131522,gen9randombattle,"{'timestamp_unix': 1752131198, 'total_turns': ...","{'p1': {'name': 'sikafenkesi', 'ladder_rating_...","{'format_type': 'random_battle', 'revelation_p...","[{'turn_number': 1, 'events': [{'seq': 0, 'typ...","{'pokemon_used': {'p1': ['p1-0', 'p1-1', 'p1-2...","{'validated': True, 'issues': [], 'sha256_raw_..."
1,1.0.0,gen9randombattle-2391190361,gen9randombattle,"{'timestamp_unix': 1750814686, 'total_turns': ...","{'p1': {'name': 'Capsubaru', 'ladder_rating_pr...","{'format_type': 'random_battle', 'revelation_p...","[{'turn_number': 1, 'events': [{'seq': 0, 'typ...","{'pokemon_used': {'p1': ['p1-0', 'p1-1', 'p1-2...","{'validated': True, 'issues': [], 'sha256_raw_..."
2,1.0.0,gen9randombattle-2395970919,gen9randombattle,"{'timestamp_unix': 1751510157, 'total_turns': ...","{'p1': {'name': 'DraconicSpeed', 'ladder_ratin...","{'format_type': 'random_battle', 'revelation_p...","[{'turn_number': 1, 'events': [{'seq': 0, 'typ...","{'pokemon_used': {'p1': ['p1-0', 'p1-1', 'p1-2...","{'validated': True, 'issues': [], 'sha256_raw_..."
3,1.0.0,gen9randombattle-2401616868,gen9randombattle,"{'timestamp_unix': 1752354639, 'total_turns': ...","{'p1': {'name': 'Ferrothornxd', 'ladder_rating...","{'format_type': 'random_battle', 'revelation_p...","[{'turn_number': 1, 'events': [{'seq': 0, 'typ...","{'pokemon_used': {'p1': ['p1-0', 'p1-1', 'p1-2...","{'validated': True, 'issues': [], 'sha256_raw_..."
4,1.0.0,gen9randombattle-2395508503,gen9randombattle,"{'timestamp_unix': 1751451540, 'total_turns': ...","{'p1': {'name': 'gonorrentoplus', 'ladder_rati...","{'format_type': 'random_battle', 'revelation_p...","[{'turn_number': 1, 'events': [{'seq': 0, 'typ...","{'pokemon_used': {'p1': ['p1-0', 'p1-1', 'p1-2...","{'validated': True, 'issues': [], 'sha256_raw_..."


In [8]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13979 entries, 0 to 13978
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   schema_version   13979 non-null  object
 1   battle_id        13979 non-null  object
 2   format_id        13979 non-null  object
 3   metadata         13979 non-null  object
 4   players          13979 non-null  object
 5   team_revelation  13979 non-null  object
 6   turns            13979 non-null  object
 7   summary          13979 non-null  object
 8   integrity        13979 non-null  object
dtypes: object(9)
memory usage: 983.0+ KB
None
       schema_version                    battle_id         format_id  \
count           13979                        13979             13979   
unique              1                        13979                 1   
top             1.0.0  gen9randombattle-2400131522  gen9randombattle   
freq            13979                            1            

In [12]:
first = df.iloc[0]
print("Claves nivel 1:", list(first.keys()))

# Mirada rápida a columnas anidadas típicas
for key in ["metadata", "players", "summary"]:
    print(f"\n== {key} ==")
    v = first.get(key)
    if isinstance(v, dict):
        print("keys:", list(v.keys())[:30])
        # muestra parte del contenido
        for k in list(v.keys())[:5]:
            print(f" - {k}: ", type(v[k]), v[k] if not isinstance(v[k], (dict, list)) else "(nested)")
    else:
        print(type(v), v)

# turns suele ser lista
turns = first.get("turns", [])
print(f"\nturns: tipo={type(turns)}, len={len(turns)}")
if turns and isinstance(turns[0], dict):
    print("keys turno[0]:", list(turns[0].keys())[:30])
    pprint(turns[0])

Claves nivel 1: ['schema_version', 'battle_id', 'format_id', 'metadata', 'players', 'team_revelation', 'turns', 'summary', 'integrity']

== metadata ==
keys: ['timestamp_unix', 'total_turns', 'outcome']
 - timestamp_unix:  <class 'int'> 1752131198
 - total_turns:  <class 'int'> 11
 - outcome:  <class 'dict'> (nested)

== players ==
keys: ['p1', 'p2']
 - p1:  <class 'dict'> (nested)
 - p2:  <class 'dict'> (nested)

== summary ==
keys: ['pokemon_used', 'fainted_order', 'major_events']
 - pokemon_used:  <class 'dict'> (nested)
 - fainted_order:  <class 'dict'> (nested)
 - major_events:  <class 'NoneType'> None

turns: tipo=<class 'list'>, len=11
keys turno[0]: ['turn_number', 'events', 'state_after']
{'events': [{'into_uid': 'p1-1',
             'player': 'p1',
             'pokemon_uid': 'p1-1',
             'seq': 0,
             'type': 'switch'},
            {'move_id': 'calmmind',
             'player': 'p2',
             'pokemon_uid': 'p2-0',
             'seq': 1,
             'ta

In [13]:
# 'first' es el dict de la primera batalla que ya mostraste
battle_preview = pd.json_normalize([first], sep=".")
# Elige columnas útiles a nivel combate
cols_battle = [
    "battle_id",
    "format_id",
    "schema_version",
    "metadata.total_turns",
    "metadata.timestamp_unix",
    # outcome (si existen estas subclaves dentro de outcome)
    "metadata.outcome.winner",
    "metadata.outcome.reason",
    # summary
    "summary.pokemon_used.p1",
    "summary.pokemon_used.p2",
    "summary.fainted_order.p1",
    "summary.fainted_order.p2",
]
battle_preview_filtered = battle_preview.reindex(columns=[c for c in cols_battle if c in battle_preview.columns])
battle_preview_filtered.T

,0


In [15]:
from collections import Counter
import pandas as pd
import json

# Asegura 'first' (usa el que ya tengas; si no existe, lo crea leyendo el JSON)
if "first" not in locals():
    with open("data/all_battles.json", "r") as f:
        _battles = json.load(f)
    first = _battles[0]

turns = first.get("turns", [])
turns_df = pd.json_normalize(turns, sep=".")

def event_type_counts(evts):
    if not isinstance(evts, list):
        return {}
    c = Counter(e.get("type") for e in evts if isinstance(e, dict))
    return dict(c)

turns_df["events_count"]   = turns_df["events"].apply(lambda x: len(x) if isinstance(x, list) else 0)
turns_df["event_types"]    = turns_df["events"].apply(event_type_counts)
turns_df["n_moves"]        = turns_df["event_types"].apply(lambda d: d.get("move", 0) if isinstance(d, dict) else 0)
turns_df["n_switch"]       = turns_df["event_types"].apply(lambda d: d.get("switch", 0) if isinstance(d, dict) else 0)
turns_df["n_stat_change"]  = turns_df["event_types"].apply(lambda d: d.get("stat_change", 0) if isinstance(d, dict) else 0)

def get_in(d, path, default=None):
    cur = d
    for k in path:
        if isinstance(cur, dict) and k in cur:
            cur = cur[k]
        else:
            return default
    return cur

turns_df["weather"]             = turns_df["state_after"].apply(lambda s: get_in(s, ["field","weather"]))
turns_df["global_conditions"]   = turns_df["state_after"].apply(lambda s: get_in(s, ["field","global_conditions"], []))
turns_df["p1_side_conditions"]  = turns_df["state_after"].apply(lambda s: get_in(s, ["sides","p1","side_conditions"], []))
turns_df["p2_side_conditions"]  = turns_df["state_after"].apply(lambda s: get_in(s, ["sides","p2","side_conditions"], []))

cols_turn = [
    "turn_number","events_count","n_moves","n_switch","n_stat_change",
    "weather","global_conditions","p1_side_conditions","p2_side_conditions"
]
turns_preview = turns_df.reindex(columns=[c for c in cols_turn if c in turns_df.columns]).sort_values("turn_number")
turns_preview.head(10)

KeyError: 'state_after'